Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from random import choices,random,randint,sample,shuffle
from copy import copy
import numpy as np
import lab9_lib
from pprint import pprint


## PARAMETERS INITIALIZATION & PROBLEM GENERATION

In [ ]:
PROBLEM_DIM = 1

GENES_PER_LOCUS = 1
GENOME_LENGHT=1000*GENES_PER_LOCUS
POP_DIM= 500
OFFSPRING_SIZE = 300
N_GENERATIONS = 1_000_000
N_GENS_WO_IMPROVEMENT_EXTINCTION = 5
MUTATION_PROB = 0.2


fitness = lab9_lib.make_problem(PROBLEM_DIM)


## INDIVIDUAL CLASS

In [ ]:
def hamming_distance(g1, g2):
    return sum([1 for i in range(GENOME_LENGHT) if g1[i] != g2[i]])

In [ ]:
class Individual:
    def __init__(self, genome):
        self.genome = genome
        self.fitness = fitness(genome)
        # self.gender=gender
    
    def get_genome(self):
        return self.genome

    def get_fitness(self):
        return self.fitness

    def update_fitness(self,upd_fit):
        self.fitness = upd_fit 
        
    def set_genome_update_fitness(self, genome):
        self.genome = genome
        self.fitness = fitness(genome)

## GA

In [ ]:

def two_cut_xover_with_mut(g1, g2):
    cut1 = randint(0, GENOME_LENGHT)
    cut2 = randint(0, GENOME_LENGHT)
    if cut1 > cut2:
        cut1, cut2 = cut2, cut1
    joined = (
        g1.get_genome()[:cut1] + g2.get_genome()[cut1:cut2] + g1.get_genome()[cut2:]
    )
    mut_prob=1/GENOME_LENGHT
    for i in range(GENOME_LENGHT):
        if random() < mut_prob:
            joined[i] = 1 - joined[i]
    return Individual(joined)


def tournament_selection(pop, size=10):
    c_ind = choices(range(len(pop)), k=size)
    selected_individuals = [pop[i] for i in c_ind]
    # print("sel", selected_individuals)
    return sorted(selected_individuals, key=lambda i: i.get_fitness(), reverse=True)[0]


def mutation(g):
    # Bit Flip Mutation
    mut_prob=1/GENOME_LENGHT
    genome = g.get_genome()
    for i in range(GENOME_LENGHT):
        if random() < mut_prob:
            genome[i] = 1 - genome[i]
    return Individual(genome)


def extinction(pop,to_keep=0):
    to_remove = np.random.choice(pop, size=(POP_DIM-to_keep) , replace=False)
    for i in to_remove:
        pop.remove(i)
    pop += [Individual(choices([0, 1], k=GENOME_LENGHT)) for _ in range(POP_DIM-to_keep)]
    return pop

def two_level_diversity_selection(pop, n_best):
    # first select individuals respect to the fitness value,
    # then choose for reproduction the most different individuals
    selected_individuals = sample(range(len(pop)),k=30)  #extract 20 indexes of individuals from pop
    selected_individuals = [pop[i] for i in selected_individuals] #get the individuals from pop

    to_compare = sorted(selected_individuals, key=lambda i: i.get_fitness(), reverse=True)[:n_best] #get the best fitness wise

    max_distance = -1
    selected = []
    for i in range(len(to_compare)):
        for j in range(i + 1, len(to_compare)):
            distance = hamming_distance(to_compare[i].get_genome(), to_compare[j].get_genome())
            if distance > max_distance:
                max_distance = distance
                selected = [to_compare[i], to_compare[j]]
    # print("max_distance", max_distance)
    return selected[0],selected[1]

 

In [203]:
# counter used to triger extinction
countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

# initial population
pop=[Individual(choices([0, 1], k=GENOME_LENGHT)) for _ in range(POP_DIM)]

best=max(pop, key=lambda i: i.get_fitness())

for gen in range(N_GENERATIONS):

    if countdown_to_extinction == 0:
        print("Extinction!")
        to_keep=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)[:10]
        pop=to_keep+extinction(pop[10:],to_keep=10) 

    for off in range(OFFSPRING_SIZE):
        
        if randint(0,1) < MUTATION_PROB:
            p=tournament_selection(pop)
            offspring = mutation(p)
            pop.append(offspring)
        else:
            p1,p2 = two_level_diversity_selection(pop, 3)
            offspring = two_cut_xover_with_mut(p1, p2)
    pop=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)[:POP_DIM]

    if pop[0].get_fitness() == best.get_fitness():
        countdown_to_extinction-=1
    else:
        countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

    best=pop[0]
    
    if best.get_fitness() == 1:
        print(f"Solution found at generation {gen}:{''.join(str(g) for g in best.get_genome())}")
        break

    print(f"Generation {gen}: {best.get_fitness():.2%}, entropy: {compute_pop_entropy(pop)}")

pprint(fitness.calls)


Generation 0: 54.70%, entropy: 2.208290327345859
Generation 1: 54.70%, entropy: 6.838750271403668
Generation 2: 54.70%, entropy: 12.527605218653298
Generation 3: 55.40%, entropy: 16.796695569682964
Generation 4: 55.50%, entropy: 18.305254747446043
Generation 5: 55.50%, entropy: 18.730858137964596
Generation 6: 55.50%, entropy: 18.730858137964596
Generation 7: 55.50%, entropy: 18.730858137964596
Generation 8: 55.50%, entropy: 18.730858137964596
Generation 9: 55.50%, entropy: 18.730858137964596
Extinction!
Generation 10: 55.80%, entropy: 2.335163816769232
Generation 11: 55.90%, entropy: 6.248283429861237
Generation 12: 55.90%, entropy: 13.685447704399143
Generation 13: 55.90%, entropy: 17.062171159877906
Generation 14: 55.90%, entropy: 17.062171159877906
Generation 15: 55.90%, entropy: 17.081229848576672
Generation 16: 55.90%, entropy: 17.12591629478242
Extinction!
Generation 17: 55.90%, entropy: 2.2305261977071282
Generation 18: 55.90%, entropy: 5.7141676418476415
Generation 19: 55.90%,